<a href="https://colab.research.google.com/github/pabloalfaro/drl-grid/blob/main/pruebas/dqn/dqn_agent_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!ls /content/gdrive/My\ Drive/Colab\ Notebooks/*.py

'/content/gdrive/My Drive/Colab Notebooks/dqn_agent.py'
'/content/gdrive/My Drive/Colab Notebooks/model.py'
'/content/gdrive/My Drive/Colab Notebooks/plot_utils.py'


In [ ]:
!cat '/content/gdrive/My Drive/Colab Notebooks/model.py'

{"nbformat":4,"nbformat_minor":0,"metadata":{"colab":{"name":"model.py","provenance":[],"authorship_tag":"ABX9TyM5eiQ8EcCYkrjcxsthOGcD"},"kernelspec":{"name":"python3","display_name":"Python 3"},"language_info":{"name":"python"}},"cells":[{"cell_type":"code","metadata":{"id":"wMeXgS8nCQC6","executionInfo":{"status":"ok","timestamp":1626971209374,"user_tz":-120,"elapsed":3756,"user":{"displayName":"PABLO ALFARO GOICOECHEA","photoUrl":"","userId":"02677138527546643857"}}},"source":["import torch\n","import torch.nn as nn\n","import torch.nn.functional as F\n","\n","class QNetwork(nn.Module):\n","    \"\"\"Actor (Policy) Model.\"\"\"\n","\n","    def __init__(self, state_size, action_size, seed, fc1_units=64, fc2_units=64):\n","        \"\"\"Initialize parameters and build model.\n","        Params\n","        ======\n","            state_size (int): Dimension of each state\n","            action_size (int): Dimension of each action\n","            seed (int): Random seed\n","            

In [ ]:
import sys
sys.path.append('/content/gdrive/My Drive/Colab Notebooks/')

import model

In [ ]:
import numpy as np
import random
from collections import namedtuple, deque

import torch
import torch.nn.functional as F
import torch.optim as optim

BUFFER_SIZE = int(1e5)  # replay buffer size (size D)
BATCH_SIZE = 64         # minibatch size (n_batch)
GAMMA = 0.99            # discount factor (gamma)
TAU = 1e-3              # for soft update of target parameters (tau)
LR = 5e-4               # learning rate (eta)
UPDATE_EVERY = 4        # how often to update the target network (C)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class Agent():
    """Interacts with and learns from the environment."""

    def __init__(self, state_size, action_size, seed):
        """Initialize an Agent object.
        
        Params
        ======
            state_size (int): dimension of each state
            action_size (int): dimension of each action
            seed (int): random seed
        """
        self.state_size = state_size
        self.action_size = action_size
        self.seed = random.seed(seed)

        # Q-Network
        self.qnetwork_local = QNetwork(state_size, action_size, seed).to(device)
        self.qnetwork_target = QNetwork(state_size, action_size, seed).to(device)
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=LR)

        # Replay memory
        self.memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE, seed)
        # Initialize time step (for updating every UPDATE_EVERY steps)
        self.t_step = 0
    
    def act(self, state, eps=0.):
        """Returns actions for given state as per current policy.
        
        Params
        ======
            state (array_like): current state
            eps (float): epsilon, for epsilon-greedy action selection
        """
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        self.qnetwork_local.eval()
        with torch.no_grad():
            action_values = self.qnetwork_local(state)
        self.qnetwork_local.train()

        # Epsilon-greedy action selection
        if random.random() > eps:
            return np.argmax(action_values.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.action_size))
        
    def step(self, state, action, reward, next_state, done):       
        # ------------------- train with mini-batch sample of experiences ------------------- #
        if len(self.memory) > BATCH_SIZE:
            # If enough samples are available in memory, get random subset and learn
            experiences = self.memory.sample()
            self.learn(experiences, GAMMA)
        
        # ------------------- update target network ----------------------------------------- #
        self.t_step = (self.t_step + 1) % UPDATE_EVERY
        if self.t_step == 0:             
            # If C (UPDATE_EVERY) steps have been reached, blend weights to the target network
            self.soft_update(self.qnetwork_local, self.qnetwork_target, TAU)

    def learn(self, experiences, gamma):
        """Update value parameters using given batch of experience tuples.
        Params
        ======
            experiences (Tuple[torch.Tensor]): tuple of (s, a, r, s', done) tuples 
            gamma (float): discount factor
        """
        states, actions, rewards, next_states, dones = experiences

        # Get max predicted Q values (for next states) from target model
        # - qnetwork_target : apply forward pass for the whole mini-batch
        # - detach : do not backpropagate
        # - max : get maximizing action for each sample of the mini-batch (dim=1)
        # - [0].unsqueeze(1) : transform output into a flat array
        Q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1)

        # Compute Q targets for current states (y)
        # - dones : detect if the episode has finished
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))

        # Get expected Q values from local model (Q(Sj, Aj, w))
        # - gather : for each sample select only the output value for action Aj
        Q_expected = self.qnetwork_local(states).gather(1, actions)

        # Optimize over (yj-Q(Sj, Aj, w))^2
        # * compute loss
        loss = F.mse_loss(Q_expected, Q_targets)
        # * minimize the loss
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()                            

    def soft_update(self, local_model, target_model, tau):
        """Soft update model parameters.
        θ_target = τ*θ_local + (1 - τ)*θ_target
        Params
        ======
            local_model (PyTorch model): weights will be copied from
            target_model (PyTorch model): weights will be copied to
            tau (float): interpolation parameter 
        """
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)


class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size, seed):
        """Initialize a ReplayBuffer object.
        Params
        ======
            action_size (int): dimension of each action
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
            seed (int): random seed
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)  
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)
    
    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
    
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)

        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).long().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
  
        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

In [ ]:
import gym
# from ddpg_agent import Agent

env = gym.make("Pendulum-v0")

agent = Agent(state_size=3, action_size=1, seed=0)

state = env.reset()
for j in range(200):
    action = agent.act(state)
    if render: env.render()
    state, reward, done, _ = env.step(action)
    if done:
        break 
        
env.close()

NameError: ignored

In [ ]:
!cp drive/MyDrive/Colab\ Notebooks/model.py .
import model

ModuleNotFoundError: ignored